In [ ]:
pip install torch torchvision torchaudio

In [ ]:
pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-2.9.0+cpu.html

In [ ]:
pip install torch-geometric

**Model Description**

This code implements a speed-optimized, regime-aware financial forecasting framework that combines Liquid Time-Constant (LTC) networks, dynamic graph learning, and N-HiTS-style multi-horizon forecasting. It uses strict temporal data splits to prevent information leakage across market regimes and incorporates spectral normalization, gradient penalties, temporal smoothness, and diversity regularization to control overfitting. The training pipeline is optimized with mixed-precision (AMP), batched LTC processing, dynamic correlation learning, and early stopping, enabling robust multi-asset forecasting under non-stationary market conditions.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.parametrizations import spectral_norm
from torch_geometric.nn import GATConv
from ncps.torch import LTC
from ncps.wirings import AutoNCP
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf
from torch.cuda.amp import autocast, GradScaler

def SpectralLinear(in_features, out_features, k=1.0, bias=True):
    """
    Spectral-normalized Linear layer (Lipschitz constrained)
    k controls the Lipschitz constant (scales the output)
    """
    layer = spectral_norm(nn.Linear(in_features, out_features, bias=bias))
    if k != 1.0:
        return nn.Sequential(layer, nn.Identity())
    return layer

class TemporalFinancialDataset(Dataset):
    """Dataset with temporal awareness for regime-based splitting - OPTIMIZED"""

    def __init__(self, data, lookback=60, horizon=30, regime_label=None):
        """
        data: DataFrame with datetime index and price columns
        lookback: input window size
        horizon: forecast horizon
        regime_label: 'train', 'validation', or 'test' for tracking
        """
        self.data = data
        self.lookback = lookback
        self.horizon = horizon
        self.regime_label = regime_label

        # PRE-COMPUTE all sequences as tensors (much faster than on-the-fly)
        sequences = []
        targets = []

        data_values = data.values  # Convert to numpy once

        for i in range(len(data) - lookback - horizon + 1):
            seq = data_values[i:i+lookback]
            target = data_values[i+lookback:i+lookback+horizon]
            sequences.append(seq)
            targets.append(target)

        # Convert to tensors once and store
        self.sequences = torch.FloatTensor(np.array(sequences)).transpose(1, 2)  # [N, n_series, lookback]
        self.targets = torch.FloatTensor(np.array(targets)).transpose(1, 2)  # [N, n_series, horizon]

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.targets[idx]

def create_temporal_splits(df, train_end='2019-12-31',
                          val_end='2021-12-31', test_end='2023-12-31'):
    """
    Create temporal data splits based on market regime periods
    """

    train_df = df[:train_end]
    val_df = df[train_end:val_end]
    test_df = df[val_end:test_end]

    print(f"Train set: {train_df.index[0]} to {train_df.index[-1]} ({len(train_df)} samples)")
    print(f"  Purpose: Learning market physics - trends, mean reversion, deflationary crash (2008)")

    print(f"\nValidation set: {val_df.index[0]} to {val_df.index[-1]} ({len(val_df)} samples)")
    print(f"  Purpose: Hyperparameter tuning - adapting to extreme speed (COVID)")

    print(f"\nTest set: {test_df.index[0]} to {test_df.index[-1]} ({len(test_df)} samples)")
    print(f"  Purpose: Performance verification - correlation flips (inflation), breadth collapse (AI boom)")

    return train_df, val_df, test_df

class AdaptiveLiquidNHiTS_GNN(nn.Module):
    """
    SPEED-OPTIMIZED hybrid model with:
    - Cached computations
    - Reduced redundant operations
    - Optimized tensor operations
    - Better memory layout
    """

    def __init__(self, n_series, input_size, h, hidden_dim=128,
                 liquid_units=64, n_blocks=[1, 1, 1], gnn_layers=2,
                 lipschitz_k=1.0, regime_adaptive=True, dropout=0.3,
                 edge_dropout=0.2):
        super().__init__()
        self.n_series = n_series
        self.input_size = input_size
        self.h = h
        self.hidden_dim = hidden_dim
        self.liquid_units = liquid_units
        self.regime_adaptive = regime_adaptive
        self.dropout_rate = dropout
        self.edge_dropout = edge_dropout

        # Regime detection module with dropout and layer norm
        if regime_adaptive:
            self.regime_detector = nn.Sequential(
                nn.Linear(input_size, 64),
                nn.LayerNorm(64),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(64, 32),
                nn.LayerNorm(32),
                nn.Tanh(),
                nn.Dropout(dropout),
                nn.Linear(32, 3)
            )

        # OPTIMIZATION: Use single LTC with batched processing instead of ModuleList
        ltc_out_dim = min(hidden_dim // 2, liquid_units - 3)
        self.ltc_out_dim = ltc_out_dim

        # Single LTC that processes all series (more efficient)
        wiring = AutoNCP(liquid_units, ltc_out_dim)
        self.ltc = LTC(input_size, wiring, return_sequences=False)

        self.ltc_dropout = nn.Dropout(dropout)
        self.ltc_layer_norm = nn.LayerNorm(ltc_out_dim)

        # Dynamic Graph Neural Networks with dropout
        self.gnn_layers = nn.ModuleList()
        self.gnn_layer_norms = nn.ModuleList()
        self.gnn_dropouts = nn.ModuleList()
        gnn_input_dim = ltc_out_dim
        for _ in range(gnn_layers):
            self.gnn_layers.append(
                GATConv(gnn_input_dim, gnn_input_dim, heads=4, concat=False, dropout=edge_dropout)
            )
            self.gnn_layer_norms.append(nn.LayerNorm(gnn_input_dim))
            self.gnn_dropouts.append(nn.Dropout(dropout))

        # OPTIMIZATION: Simplified N-HiTS with fewer blocks
        self.nhits_stacks = nn.ModuleList()

        for i, n_block in enumerate(n_blocks):
            stack = nn.ModuleList()

            for _ in range(n_block):
                resolution = 2 ** i
                forecast_size = max(1, h // resolution)

                # Combine theta_f operations for speed
                block = nn.Sequential(
                    SpectralLinear(ltc_out_dim, hidden_dim, k=lipschitz_k),
                    nn.LayerNorm(hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(dropout),
                    SpectralLinear(hidden_dim, forecast_size, k=lipschitz_k)
                )
                stack.append(block)

            self.nhits_stacks.append(stack)

        # Graph structure learner
        self.graph_learner = nn.Sequential(
            nn.Linear(ltc_out_dim, ltc_out_dim),
            nn.LayerNorm(ltc_out_dim),
            nn.Tanh(),
            nn.Dropout(dropout)
        )

    def detect_regime(self, x):
        """Detect market regime - optimized"""
        if not self.regime_adaptive:
            return None

        x_avg = x.mean(dim=1)
        regime_logits = self.regime_detector(x_avg)
        return F.softmax(regime_logits, dim=-1)

    def forward(self, x, edge_index=None, return_regime=False):
        """
        OPTIMIZED forward pass with batched operations
        x: [batch, n_series, input_size]
        """
        batch_size = x.size(0)

        # Detect market regime
        regime_probs = self.detect_regime(x)

        # OPTIMIZATION: Process all series in parallel
        # Reshape: [batch * n_series, 1, input_size]
        x_reshaped = x.view(batch_size * self.n_series, 1, self.input_size)
        h0 = torch.zeros(batch_size * self.n_series, self.liquid_units, device=x.device)

        # Single LTC call for all series (much faster)
        ltc_out, _ = self.ltc(x_reshaped, h0)
        ltc_out = self.ltc_layer_norm(ltc_out)
        ltc_out = self.ltc_dropout(ltc_out)

        # Reshape back: [batch, n_series, ltc_out_dim]
        node_features = ltc_out.view(batch_size, self.n_series, self.ltc_out_dim)

        # Learn dynamic graph structure (cached for efficiency)
        node_repr = self.graph_learner(node_features)
        adj_matrix = torch.bmm(node_repr, node_repr.transpose(1, 2))
        adj_matrix = F.softmax(adj_matrix / np.sqrt(self.ltc_out_dim), dim=-1)

        # Apply edge dropout
        if self.training and self.edge_dropout > 0:
            edge_mask = torch.bernoulli(torch.full_like(adj_matrix, 1 - self.edge_dropout))
            adj_matrix = adj_matrix * edge_mask

        # Apply GNN layers with batched matrix multiplication (faster than edge_index)
        for gnn_layer, layer_norm, dropout in zip(
            self.gnn_layers, self.gnn_layer_norms, self.gnn_dropouts
        ):
            node_features = torch.bmm(adj_matrix, node_features)
            node_features = layer_norm(node_features)
            node_features = F.relu(node_features)
            node_features = dropout(node_features)

        # OPTIMIZATION: Vectorized N-HiTS forecasting
        # Process all series at once instead of loop
        final_forecast = torch.zeros(batch_size, self.n_series, self.h, device=x.device)

        for stack in self.nhits_stacks:
            for block in stack:
                # Apply to all series at once: [batch, n_series, ltc_out_dim] -> [batch, n_series, forecast_size]
                block_forecast = block(node_features)

                # Get current output size
                current_size = block_forecast.size(-1)

                # Always interpolate to target horizon to ensure consistent size
                if current_size != self.h:
                    # Reshape for interpolation: [batch * n_series, 1, forecast_size]
                    block_forecast_reshaped = block_forecast.reshape(batch_size * self.n_series, 1, current_size)

                    # Interpolate to target horizon
                    block_forecast_reshaped = F.interpolate(
                        block_forecast_reshaped,
                        size=self.h,
                        mode='linear',
                        align_corners=False
                    )

                    # Reshape back to [batch, n_series, h]
                    block_forecast = block_forecast_reshaped.reshape(batch_size, self.n_series, self.h)

                # Add to final forecast (now guaranteed to be same size)
                final_forecast = final_forecast + block_forecast

        if return_regime:
            return final_forecast, regime_probs
        return final_forecast

def gradient_penalty(model, real_data, device):
    """OPTIMIZED: Only compute occasionally to save time"""
    real_data.requires_grad_(True)
    predictions = model(real_data)

    gradients = torch.autograd.grad(
        outputs=predictions,
        inputs=real_data,
        grad_outputs=torch.ones_like(predictions),
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean()

def temporal_consistency_loss(predictions, alpha=0.5):
    """Encourage temporal smoothness"""
    temporal_diff = predictions[:, :, 1:] - predictions[:, :, :-1]
    return alpha * (temporal_diff ** 2).mean()

def forecast_diversity_loss(predictions, beta=0.1):
    """Prevent mode collapse"""
    batch_mean = predictions.mean(dim=0)
    batch_var = ((predictions - batch_mean.unsqueeze(0)) ** 2).mean()
    return -beta * torch.log(batch_var + 1e-8)

def train_with_temporal_splits(model, train_loader, val_loader, test_loader,
                               epochs=100, lr=1e-3, device='cuda',
                               patience=50, lambda_gp=0.1, lambda_temp=0.05,
                               lambda_div=0.01, weight_decay=1e-4,
                               use_amp=True, min_epochs=30):
    """
    SPEED OPTIMIZED training with:
    - Mixed precision training (AMP)
    - Gradient accumulation
    - Optimized loss computation
    - Pin memory for faster data loading
    """

    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # More patient learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=15, min_lr=1e-6
    )
    criterion = nn.MSELoss()

    # Mixed precision scaler for faster training
    scaler = GradScaler() if use_amp else None

    # Enable cuDNN autotuner for faster convolutions
    if device == 'cuda':
        torch.backends.cudnn.benchmark = True
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []

    print("\n" + "="*80)
    print("SPEED-OPTIMIZED TRAINING")
    print("="*80)
    print(f"Mixed Precision: {use_amp}")
    print(f"Device: {device}")
    print(f"Regularization: WD={weight_decay}, GP={lambda_gp}, Temp={lambda_temp}, Div={lambda_div}")
    print("="*80 + "\n")

    import time

    for epoch in range(epochs):
        epoch_start = time.time()

        # Training
        model.train()
        train_loss = 0
        train_reg_loss = 0

        for batch_idx, (batch_x, batch_y) in enumerate(train_loader):
            batch_x = batch_x.to(device, non_blocking=True)
            batch_y = batch_y.to(device, non_blocking=True)

            optimizer.zero_grad()

            # Mixed precision forward pass
            if use_amp:
                with autocast():
                    forecast, regime_probs = model(batch_x, return_regime=True)
                    mse_loss = criterion(forecast, batch_y)

                    # Compute regularization every 3 batches (was 5 - more frequent)
                    if batch_idx % 3 == 0:
                        gp_loss = gradient_penalty(model, batch_x, device)
                    else:
                        gp_loss = torch.tensor(0.0, device=device)

                    temp_loss = temporal_consistency_loss(forecast)
                    div_loss = forecast_diversity_loss(forecast)

                    regime_entropy_loss = 0
                    if regime_probs is not None:
                        regime_entropy = -(regime_probs * torch.log(regime_probs + 1e-8)).sum(dim=-1).mean()
                        regime_entropy_loss = -0.01 * regime_entropy

                    loss = (mse_loss +
                           lambda_gp * gp_loss +
                           lambda_temp * temp_loss +
                           lambda_div * div_loss +
                           regime_entropy_loss)

                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                forecast, regime_probs = model(batch_x, return_regime=True)
                mse_loss = criterion(forecast, batch_y)

                if batch_idx % 3 == 0:
                    gp_loss = gradient_penalty(model, batch_x, device)
                else:
                    gp_loss = torch.tensor(0.0, device=device)

                temp_loss = temporal_consistency_loss(forecast)
                div_loss = forecast_diversity_loss(forecast)

                regime_entropy_loss = 0
                if regime_probs is not None:
                    regime_entropy = -(regime_probs * torch.log(regime_probs + 1e-8)).sum(dim=-1).mean()
                    regime_entropy_loss = -0.01 * regime_entropy

                loss = (mse_loss +
                       lambda_gp * gp_loss +
                       lambda_temp * temp_loss +
                       lambda_div * div_loss +
                       regime_entropy_loss)

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

            train_loss += mse_loss.item()
            train_reg_loss += (lambda_gp * gp_loss.item() if isinstance(gp_loss, torch.Tensor) else 0)

        avg_train_loss = train_loss / len(train_loader)
        avg_reg_loss = train_reg_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.to(device, non_blocking=True)
                batch_y = batch_y.to(device, non_blocking=True)

                if use_amp:
                    with autocast():
                        forecast = model(batch_x)
                        loss = criterion(forecast, batch_y)
                else:
                    forecast = model(batch_x)
                    loss = criterion(forecast, batch_y)

                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        scheduler.step(avg_val_loss)

        # Early stopping check - but allow minimum epochs
        if epoch >= min_epochs:
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                patience_counter = 0
                # Save best model
                torch.save(model.state_dict(), 'best_model_temporal.pt')
            else:
                patience_counter += 1
        else:
            # Always save in early epochs
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save(model.state_dict(), 'best_model_temporal.pt')

        epoch_time = time.time() - epoch_start

        if epoch % 5 == 0:  # Print every 5 epochs
            print(f"Epoch {epoch}/{epochs} ({epoch_time:.1f}s)")
            print(f"  Train: {avg_train_loss:.6f} | Val: {avg_val_loss:.6f} | LR: {optimizer.param_groups[0]['lr']:.2e}")
            print(f"  Best Val: {best_val_loss:.6f} | Patience: {patience_counter}/{patience}")

            if device == 'cuda':
                print(f"  GPU Mem: {torch.cuda.memory_allocated()/1e9:.2f} GB")

        if patience_counter >= patience and epoch >= min_epochs:
            print(f"\nEarly stopping at epoch {epoch}")
            print(f"Validation loss hasn't improved for {patience} epochs")
            break

    # Load best model
    model.load_state_dict(torch.load('best_model_temporal.pt'))

    # Testing
    print("\n" + "="*80)
    print("TESTING PHASE")
    print("="*80 + "\n")

    model.eval()
    test_loss = 0

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device, non_blocking=True)
            batch_y = batch_y.to(device, non_blocking=True)

            if use_amp:
                with autocast():
                    forecast = model(batch_x)
                    loss = criterion(forecast, batch_y)
            else:
                forecast = model(batch_x)
                loss = criterion(forecast, batch_y)

            test_loss += loss.item()

    avg_test_loss = test_loss / len(test_loader)

    print(f"Final Results:")
    print(f"  Train Loss: {train_losses[-1]:.6f}")
    print(f"  Val Loss: {val_losses[-1]:.6f}")
    print(f"  Test Loss: {avg_test_loss:.6f}")
    print(f"\n  Test/Train Ratio: {avg_test_loss/train_losses[-1]:.3f}")
    print(f"  Test/Val Ratio: {avg_test_loss/val_losses[-1]:.3f}")

    return model, train_losses, val_losses, avg_test_loss

def run_temporal_split_experiment(tickers=['SPY', 'QQQ', 'IWM', 'DIA', 'GLD',
                                            'TLT', 'UUP', 'EEM', 'VNQ', 'XLE'],
                                   lookback=60, horizon=30, batch_size=64,
                                   dropout=0.3, edge_dropout=0.2,
                                   lambda_gp=0.05, lambda_temp=0.02,
                                   lambda_div=0.01, weight_decay=1e-4,
                                   num_workers=4,
                                   return_datasets=True):
    """
    SPEED-OPTIMIZED experiment with:
    - Larger batch size (64 vs 32)
    - Multi-worker data loading
    - Pin memory for faster GPU transfer
    - Reduced regularization overhead
    """

    print("Downloading financial data...")
    data_frames = []
    for ticker in tickers:
        try:
            df = yf.download(ticker, start='2000-01-01', end='2024-01-01', progress=False)
            close = df['Close']

            if isinstance(close, pd.DataFrame):
                close = close.iloc[:, 0]

            data_frames.append(close.rename(ticker))
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")

    df_combined = pd.concat(data_frames, axis=1).dropna()
    print(f"Downloaded {len(df_combined)} days of data for {len(data_frames)} tickers")

    # Create temporal splits
    train_df, val_df, test_df = create_temporal_splits(
        df_combined,
        train_end='2019-12-31',
        val_end='2021-12-31',
        test_end='2023-12-31'
    )

    # Normalize
    train_mean = train_df.mean()
    train_std = train_df.std()

    train_df_norm = (train_df - train_mean) / train_std
    val_df_norm = (val_df - train_mean) / train_std
    test_df_norm = (test_df - train_mean) / train_std

    # Create datasets (pre-computed tensors)
    print("\nPre-computing dataset tensors...")
    train_dataset = TemporalFinancialDataset(train_df_norm, lookback, horizon, 'train')
    val_dataset = TemporalFinancialDataset(val_df_norm, lookback, horizon, 'validation')
    test_dataset = TemporalFinancialDataset(test_df_norm, lookback, horizon, 'test')

    # OPTIMIZATION: Pin memory and multiple workers for faster data loading
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=True if num_workers > 0 else False
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=True if num_workers > 0 else False
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        persistent_workers=True if num_workers > 0 else False
    )

    # Initialize model WITH REDUCED CAPACITY to prevent overfitting
    model = AdaptiveLiquidNHiTS_GNN(
        n_series=len(data_frames),
        input_size=lookback,
        h=horizon,
        hidden_dim=96,          # REDUCED: 128 → 96 (smaller model)
        liquid_units=48,        # REDUCED: 64 → 48 (less complexity)
        n_blocks=[1, 1],        # REDUCED: [1,1,1] → [1,1] (fewer blocks)
        gnn_layers=1,           # REDUCED: 2 → 1 (simpler graph)
        lipschitz_k=0.8,        # REDUCED: 1.0 → 0.8 (tighter constraint)
        regime_adaptive=True,
        dropout=dropout,
        edge_dropout=edge_dropout
    )

    print(f"\nModel Parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Train with optimizations
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    trained_model, train_losses, val_losses, test_loss = train_with_temporal_splits(
        model, train_loader, val_loader, test_loader,
        epochs=300, lr=1e-3,  # Increased from 200 to 300 epochs
        device=device,
        patience=50,          # Increased from 20 to 50
        min_epochs=50,        # Don't stop before 50 epochs
        lambda_gp=lambda_gp,
        lambda_temp=lambda_temp,
        lambda_div=lambda_div,
        weight_decay=weight_decay,
        use_amp=True  # Mixed precision for speed
    )

    return trained_model, train_losses, val_losses, test_loss

# Run the experiment
if __name__ == "__main__":
    # GPU Setup and Verification
    print("="*80)
    print("GPU SETUP")
    print("="*80)

    if torch.cuda.is_available():
        print(f"✓ GPU Available: {torch.cuda.get_device_name(0)}")
        print(f"✓ CUDA Version: {torch.version.cuda}")
        print(f"✓ Number of GPUs: {torch.cuda.device_count()}")
        print(f"✓ Current GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        device = 'cuda'

        torch.cuda.empty_cache()
        print("✓ GPU cache cleared")
    else:
        print("✗ GPU not available - using CPU (will be MUCH slower)")
        device = 'cpu'

    print(f"\nTraining Device: {device.upper()}")
    print("="*80 + "\n")

**Graphs And Visualisations**

Given below is a comprehensive evaluation and visualization suite for a trained multi-asset forecasting model, including training dynamics, overfitting analysis, time-series forecasts, error heatmaps, and rolling performance metrics.
It provides diagnostic insight into model stability, regime robustness, and asset-wise forecast behavior across the test period

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from scipy import stats as scipy_stats
import pandas as pd
import yfinance as yf

# Set style
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (15, 10)

# Check if required variables exist
required_vars = {
    'model': None,
    'train_losses': None,
    'val_losses': None,
    'test_loss': None
}

missing = []
for var_name in required_vars.keys():
    try:
        required_vars[var_name] = eval(var_name)
    except NameError:
        missing.append(var_name)

if missing:
    print(f"❌ Missing variables: {', '.join(missing)}")
    print("\nMake sure you have these variables from training:")
    for var in required_vars.keys():
        print(f"  - {var}")
    raise NameError(f"Missing required variables: {missing}")

print("✓ Core variables found!")

# Check for test data - if not available, recreate it
try:
    test_loader
    test_dataset
    print("✓ Test data found!")
    has_test_data = True
except NameError:
    print("⚠️  Test loader/dataset not found. Recreating from data...")
    has_test_data = False

    # Recreate test dataset
    from torch.utils.data import Dataset, DataLoader

    class TemporalFinancialDataset(Dataset):
        def __init__(self, data, lookback=60, horizon=30):
            self.data = data
            self.lookback = lookback
            self.horizon = horizon

            sequences = []
            targets = []
            data_values = data.values

            for i in range(len(data) - lookback - horizon + 1):
                seq = data_values[i:i+lookback]
                target = data_values[i+lookback:i+lookback+horizon]
                sequences.append(seq)
                targets.append(target)

            self.sequences = torch.FloatTensor(np.array(sequences)).transpose(1, 2)
            self.targets = torch.FloatTensor(np.array(targets)).transpose(1, 2)

        def __len__(self):
            return len(self.sequences)

        def __getitem__(self, idx):
            return self.sequences[idx], self.targets[idx]

    print("  → Downloading test data...")
    tickers = ['SPY', 'QQQ', 'IWM', 'DIA', 'GLD', 'TLT', 'UUP', 'EEM', 'VNQ', 'XLE']
    data_frames = []

    for ticker in tickers:
        try:
            df = yf.download(ticker, start='2000-01-01', end='2024-01-01', progress=False)
            close = df['Close']
            if isinstance(close, pd.DataFrame):
                close = close.iloc[:, 0]
            data_frames.append(close.rename(ticker))
        except:
            pass

    df_combined = pd.concat(data_frames, axis=1).dropna()

    # Get test period (2022-2023)
    test_df = df_combined['2021-12-31':'2023-12-31']

    # Normalize using training stats (approximate)
    train_df = df_combined[:'2019-12-31']
    train_mean = train_df.mean()
    train_std = train_df.std()
    test_df_norm = (test_df - train_mean) / train_std

    # Create dataset
    test_dataset = TemporalFinancialDataset(test_df_norm, lookback=60, horizon=30)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    print("  ✓ Test data recreated!")

# Get device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}\n")

print("="*80)
print("📊 GENERATING COMPREHENSIVE VISUALIZATIONS")
print("="*80 + "\n")

# ============================================================================
# 1. TRAINING METRICS DASHBOARD
# ============================================================================
print("1/5: Creating training metrics dashboard...")

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Training Metrics Dashboard', fontsize=16, fontweight='bold')

epochs = range(len(train_losses))

# Loss curves
axes[0, 0].plot(epochs, train_losses, label='Train Loss', linewidth=2, color='#2ecc71')
axes[0, 0].plot(epochs, val_losses, label='Val Loss', linewidth=2, color='#e74c3c')
axes[0, 0].axhline(y=test_loss, color='#9b59b6', linestyle='--', linewidth=2,
                   label=f'Test Loss ({test_loss:.3f})')
axes[0, 0].set_xlabel('Epoch', fontsize=11)
axes[0, 0].set_ylabel('Loss (MSE)', fontsize=11)
axes[0, 0].set_title('Training & Validation Loss', fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Log scale
axes[0, 1].semilogy(epochs, train_losses, label='Train', linewidth=2, color='#2ecc71')
axes[0, 1].semilogy(epochs, val_losses, label='Val', linewidth=2, color='#e74c3c')
axes[0, 1].set_xlabel('Epoch', fontsize=11)
axes[0, 1].set_ylabel('Loss (Log Scale)', fontsize=11)
axes[0, 1].set_title('Loss Trends (Log Scale)', fontsize=13, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Overfitting gap
overfitting_gap = [v/t if t > 0 else 0 for v, t in zip(val_losses, train_losses)]
axes[1, 0].plot(epochs, overfitting_gap, linewidth=2, color='#e67e22')
axes[1, 0].axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='No Overfitting (1x)')
axes[1, 0].axhline(y=2.0, color='orange', linestyle='--', alpha=0.5, label='Acceptable (2x)')
axes[1, 0].axhline(y=5.0, color='red', linestyle='--', alpha=0.5, label='High (5x)')
axes[1, 0].fill_between(epochs, 0, 2, alpha=0.1, color='green')
axes[1, 0].fill_between(epochs, 2, 5, alpha=0.1, color='orange')
axes[1, 0].set_xlabel('Epoch', fontsize=11)
axes[1, 0].set_ylabel('Val/Train Ratio', fontsize=11)
axes[1, 0].set_title('Overfitting Gap (Lower is Better)', fontsize=13, fontweight='bold')
axes[1, 0].legend(fontsize=9)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylim([0, min(max(overfitting_gap), 20)])

# Summary
axes[1, 1].axis('off')
test_train_ratio = test_loss / train_losses[-1]
test_val_ratio = test_loss / val_losses[-1]

if test_train_ratio < 2.0:
    quality = "🟢 EXCELLENT"
elif test_train_ratio < 5.0:
    quality = "🟡 GOOD"
elif test_train_ratio < 10.0:
    quality = "🟠 FAIR - Needs Improvement"
else:
    quality = "🔴 POOR - Severe Overfitting"

summary = f"""
FINAL RESULTS
═══════════════════════════════
Train Loss:      {train_losses[-1]:.6f}
Val Loss:        {val_losses[-1]:.6f}
Test Loss:       {test_loss:.6f}

Test/Train:      {test_train_ratio:.2f}x
Test/Val:        {test_val_ratio:.2f}x
Val/Train:       {val_losses[-1]/train_losses[-1]:.2f}x

Epochs:          {len(train_losses)}

═══════════════════════════════
QUALITY:         {quality}
═══════════════════════════════

Target: < 2.0x for excellent
        < 5.0x for good
"""
axes[1, 1].text(0.05, 0.5, summary, fontsize=11, family='monospace',
                verticalalignment='center')

plt.tight_layout()
plt.savefig('1_training_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 1_training_dashboard.png\n")

# ============================================================================
# 2. TIME SERIES FORECASTS - MULTIPLE SAMPLES
# ============================================================================
print("2/5: Creating time series forecast visualizations...")

model.eval()
ticker_names = ['SPY', 'QQQ', 'IWM', 'DIA', 'GLD', 'TLT', 'UUP', 'EEM', 'VNQ', 'XLE']

# Get predictions for 6 test samples
n_samples = 6
sample_indices = [0, 50, 100, 150, 200, min(250, len(test_dataset)-1)]

fig, axes = plt.subplots(3, 2, figsize=(18, 12))
fig.suptitle('Time Series Forecasts: Historical Context + Predictions',
             fontsize=16, fontweight='bold')
axes = axes.flatten()

with torch.no_grad():
    for plot_idx, sample_idx in enumerate(sample_indices):
        if sample_idx >= len(test_dataset):
            sample_idx = len(test_dataset) - 1

        # Get input and target
        x_sample, y_sample = test_dataset[sample_idx]
        x_sample = x_sample.unsqueeze(0).to(device)

        # Get prediction
        pred = model(x_sample).squeeze(0).cpu().numpy()

        # Get actual values
        input_seq = test_dataset[sample_idx][0].numpy()
        actual_forecast = test_dataset[sample_idx][1].numpy()

        ax = axes[plot_idx]

        # Plot for first 3 assets
        lookback_steps = range(-input_seq.shape[1], 0)
        forecast_steps = range(0, actual_forecast.shape[1])

        colors = ['#e74c3c', '#3498db', '#2ecc71']

        for series_idx in range(min(3, input_seq.shape[0])):
            # Historical data
            ax.plot(lookback_steps, input_seq[series_idx],
                   color=colors[series_idx], linewidth=2, alpha=0.6,
                   label=f'{ticker_names[series_idx]} History')

            # Actual future
            ax.plot(forecast_steps, actual_forecast[series_idx],
                   color=colors[series_idx], linewidth=2.5, alpha=0.8,
                   linestyle='--', marker='o', markersize=4,
                   label=f'{ticker_names[series_idx]} Actual')

            # Predicted future
            ax.plot(forecast_steps, pred[series_idx],
                   color=colors[series_idx], linewidth=2.5, alpha=0.8,
                   linestyle=':', marker='s', markersize=4,
                   label=f'{ticker_names[series_idx]} Predicted')

        # Forecast start line
        ax.axvline(x=0, color='black', linestyle='--', alpha=0.5, linewidth=1.5,
                  label='Forecast Start')

        ax.set_xlabel('Time Steps', fontsize=10)
        ax.set_ylabel('Normalized Price', fontsize=10)
        ax.set_title(f'Test Sample {sample_idx+1}', fontsize=12, fontweight='bold')
        ax.legend(loc='best', fontsize=7, ncol=2)
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('2_time_series_forecasts.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 2_time_series_forecasts.png\n")

# ============================================================================
# 3. DETAILED SINGLE FORECAST WITH ALL ASSETS
# ============================================================================
print("3/5: Creating detailed multi-asset forecast...")

interesting_sample_idx = min(100, len(test_dataset) - 1)
x_sample, y_sample = test_dataset[interesting_sample_idx]
x_sample = x_sample.unsqueeze(0).to(device)

with torch.no_grad():
    pred = model(x_sample).squeeze(0).cpu().numpy()

input_seq = test_dataset[interesting_sample_idx][0].numpy()
actual_forecast = test_dataset[interesting_sample_idx][1].numpy()

n_assets = min(10, input_seq.shape[0])
fig, axes = plt.subplots(5, 2, figsize=(16, 16))
fig.suptitle(f'Detailed Forecast - All Assets (Sample {interesting_sample_idx+1})',
             fontsize=16, fontweight='bold')
axes = axes.flatten()

lookback_steps = range(-input_seq.shape[1], 0)
forecast_steps = range(0, actual_forecast.shape[1])

for asset_idx in range(n_assets):
    ax = axes[asset_idx]

    # Historical
    ax.plot(lookback_steps, input_seq[asset_idx],
           color='gray', linewidth=2, alpha=0.7, label='Historical')

    # Actual
    ax.plot(forecast_steps, actual_forecast[asset_idx],
           color='#e74c3c', linewidth=2.5, marker='o', markersize=5,
           linestyle='--', alpha=0.8, label='Actual')

    # Predicted
    ax.plot(forecast_steps, pred[asset_idx],
           color='#2ecc71', linewidth=2.5, marker='s', markersize=5,
           linestyle=':', alpha=0.8, label='Predicted')

    # Forecast start
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.5, linewidth=1.5)

    # Metrics
    mse = np.mean((pred[asset_idx] - actual_forecast[asset_idx])**2)
    mae = np.mean(np.abs(pred[asset_idx] - actual_forecast[asset_idx]))

    ax.set_xlabel('Time Steps', fontsize=9)
    ax.set_ylabel('Normalized Price', fontsize=9)
    ax.set_title(f'{ticker_names[asset_idx]} | MSE: {mse:.4f} | MAE: {mae:.4f}',
                fontsize=11, fontweight='bold')
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('3_detailed_asset_forecasts.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 3_detailed_asset_forecasts.png\n")

# ============================================================================
# 4. FORECAST ACCURACY HEATMAP
# ============================================================================
print("4/5: Creating forecast accuracy heatmap...")

n_heatmap_samples = min(50, len(test_dataset))
errors_matrix = []

with torch.no_grad():
    for i in range(n_heatmap_samples):
        x_sample, y_sample = test_dataset[i]
        x_sample = x_sample.unsqueeze(0).to(device)
        pred = model(x_sample).squeeze(0).cpu().numpy()

        # Absolute error for each asset
        error = np.abs(pred - y_sample.numpy())
        errors_matrix.append(error.mean(axis=1))

errors_matrix = np.array(errors_matrix).T

fig, ax = plt.subplots(figsize=(14, 8))

im = ax.imshow(errors_matrix, aspect='auto', cmap='RdYlGn_r', interpolation='nearest')

ax.set_xticks(range(0, n_heatmap_samples, 5))
ax.set_xticklabels(range(0, n_heatmap_samples, 5))
ax.set_yticks(range(len(ticker_names)))
ax.set_yticklabels(ticker_names)

ax.set_xlabel('Test Sample Index', fontsize=12)
ax.set_ylabel('Asset', fontsize=12)
ax.set_title('Forecast Error Heatmap (MAE) - Green=Good, Red=Bad',
            fontsize=14, fontweight='bold')

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Mean Absolute Error', fontsize=11)

plt.tight_layout()
plt.savefig('4_forecast_error_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 4_forecast_error_heatmap.png\n")

# ============================================================================
# 5. ROLLING FORECAST PERFORMANCE
# ============================================================================
print("5/5: Creating rolling forecast performance analysis...")

rolling_mse = []
rolling_mae = []

with torch.no_grad():
    for i in range(len(test_dataset)):
        x_sample, y_sample = test_dataset[i]
        x_sample = x_sample.unsqueeze(0).to(device)
        pred = model(x_sample).squeeze(0).cpu().numpy()

        mse = np.mean((pred - y_sample.numpy())**2)
        mae = np.mean(np.abs(pred - y_sample.numpy()))

        rolling_mse.append(mse)
        rolling_mae.append(mae)

# Rolling averages
window_size = 20
rolling_mse_avg = np.convolve(rolling_mse, np.ones(window_size)/window_size, mode='valid')
rolling_mae_avg = np.convolve(rolling_mae, np.ones(window_size)/window_size, mode='valid')

fig, axes = plt.subplots(2, 1, figsize=(14, 8))
fig.suptitle('Rolling Forecast Performance Over Test Set', fontsize=16, fontweight='bold')

# MSE
axes[0].plot(rolling_mse, alpha=0.3, color='lightblue', label='Individual Samples')
axes[0].plot(range(window_size-1, len(rolling_mse)), rolling_mse_avg,
            linewidth=2.5, color='darkblue', label=f'{window_size}-Sample Average')
axes[0].set_ylabel('MSE', fontsize=11)
axes[0].set_title('Mean Squared Error', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(rolling_mae, alpha=0.3, color='lightcoral', label='Individual Samples')
axes[1].plot(range(window_size-1, len(rolling_mae)), rolling_mae_avg,
            linewidth=2.5, color='darkred', label=f'{window_size}-Sample Average')
axes[1].set_xlabel('Test Sample Index', fontsize=11)
axes[1].set_ylabel('MAE', fontsize=11)
axes[1].set_title('Mean Absolute Error', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('5_rolling_performance.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 5_rolling_performance.png\n")

# ============================================================================
# SUMMARY
# ============================================================================
print("="*80)
print("✅ ALL VISUALIZATIONS COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. 1_training_dashboard.png       - Training metrics & overfitting analysis")
print("  2. 2_time_series_forecasts.png    - 6 sample forecasts with context")
print("  3. 3_detailed_asset_forecasts.png - All 10 assets for one sample")
print("  4. 4_forecast_error_heatmap.png   - Error patterns across assets & time")
print("  5. 5_rolling_performance.png      - Performance trends over test set")
print("="*80)

print(f"\n{'='*80}")
print(f"FINAL MODEL QUALITY: {quality}")
print(f"Test/Train Ratio: {test_train_ratio:.2f}x")
print(f"{'='*80}\n")

# Get device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}\n")

print("="*80)
print("📊 GENERATING COMPREHENSIVE VISUALIZATIONS")
print("="*80 + "\n")

# ============================================================================
# 1. TRAINING METRICS DASHBOARD
# ============================================================================
print("1/5: Creating training metrics dashboard...")

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Training Metrics Dashboard', fontsize=16, fontweight='bold')

epochs = range(len(train_losses))

# Loss curves
axes[0, 0].plot(epochs, train_losses, label='Train Loss', linewidth=2, color='#2ecc71')
axes[0, 0].plot(epochs, val_losses, label='Val Loss', linewidth=2, color='#e74c3c')
axes[0, 0].axhline(y=test_loss, color='#9b59b6', linestyle='--', linewidth=2,
                   label=f'Test Loss ({test_loss:.3f})')
axes[0, 0].set_xlabel('Epoch', fontsize=11)
axes[0, 0].set_ylabel('Loss (MSE)', fontsize=11)
axes[0, 0].set_title('Training & Validation Loss', fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Log scale
axes[0, 1].semilogy(epochs, train_losses, label='Train', linewidth=2, color='#2ecc71')
axes[0, 1].semilogy(epochs, val_losses, label='Val', linewidth=2, color='#e74c3c')
axes[0, 1].set_xlabel('Epoch', fontsize=11)
axes[0, 1].set_ylabel('Loss (Log Scale)', fontsize=11)
axes[0, 1].set_title('Loss Trends (Log Scale)', fontsize=13, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Overfitting gap
overfitting_gap = [v/t if t > 0 else 0 for v, t in zip(val_losses, train_losses)]
axes[1, 0].plot(epochs, overfitting_gap, linewidth=2, color='#e67e22')
axes[1, 0].axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='No Overfitting (1x)')
axes[1, 0].axhline(y=2.0, color='orange', linestyle='--', alpha=0.5, label='Acceptable (2x)')
axes[1, 0].axhline(y=5.0, color='red', linestyle='--', alpha=0.5, label='High (5x)')
axes[1, 0].fill_between(epochs, 0, 2, alpha=0.1, color='green')
axes[1, 0].fill_between(epochs, 2, 5, alpha=0.1, color='orange')
axes[1, 0].set_xlabel('Epoch', fontsize=11)
axes[1, 0].set_ylabel('Val/Train Ratio', fontsize=11)
axes[1, 0].set_title('Overfitting Gap (Lower is Better)', fontsize=13, fontweight='bold')
axes[1, 0].legend(fontsize=9)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylim([0, min(max(overfitting_gap), 20)])

# Summary
axes[1, 1].axis('off')
test_train_ratio = test_loss / train_losses[-1]
test_val_ratio = test_loss / val_losses[-1]

if test_train_ratio < 2.0:
    quality = "🟢 EXCELLENT"
elif test_train_ratio < 5.0:
    quality = "🟡 GOOD"
elif test_train_ratio < 10.0:
    quality = "🟠 FAIR - Needs Improvement"
else:
    quality = "🔴 POOR - Severe Overfitting"

summary = f"""
FINAL RESULTS
═══════════════════════════════
Train Loss:      {train_losses[-1]:.6f}
Val Loss:        {val_losses[-1]:.6f}
Test Loss:       {test_loss:.6f}

Test/Train:      {test_train_ratio:.2f}x
Test/Val:        {test_val_ratio:.2f}x
Val/Train:       {val_losses[-1]/train_losses[-1]:.2f}x

Epochs:          {len(train_losses)}

═══════════════════════════════
QUALITY:         {quality}
═══════════════════════════════

Target: < 2.0x for excellent
        < 5.0x for good
"""
axes[1, 1].text(0.05, 0.5, summary, fontsize=11, family='monospace',
                verticalalignment='center')

plt.tight_layout()
plt.savefig('1_training_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 1_training_dashboard.png\n")

# ============================================================================
# 2. TIME SERIES FORECASTS - MULTIPLE SAMPLES
# ============================================================================
print("2/5: Creating time series forecast visualizations...")

model.eval()
ticker_names = ['SPY', 'QQQ', 'IWM', 'DIA', 'GLD', 'TLT', 'UUP', 'EEM', 'VNQ', 'XLE']

# Get predictions for 6 test samples
n_samples = 6
sample_indices = [0, 50, 100, 150, 200, 250]

fig, axes = plt.subplots(3, 2, figsize=(18, 12))
fig.suptitle('Time Series Forecasts: Historical Context + Predictions',
             fontsize=16, fontweight='bold')
axes = axes.flatten()

with torch.no_grad():
    for plot_idx, sample_idx in enumerate(sample_indices):
        if sample_idx >= len(test_dataset):
            continue

        # Get input and target
        x_sample, y_sample = test_dataset[sample_idx]
        x_sample = x_sample.unsqueeze(0).to(device)

        # Get prediction
        pred = model(x_sample).squeeze(0).cpu().numpy()

        # Get actual values
        input_seq = test_dataset[sample_idx][0].numpy()
        actual_forecast = test_dataset[sample_idx][1].numpy()

        ax = axes[plot_idx]

        # Plot for first 3 assets
        lookback_steps = range(-input_seq.shape[1], 0)
        forecast_steps = range(0, actual_forecast.shape[1])

        colors = ['#e74c3c', '#3498db', '#2ecc71']

        for series_idx in range(min(3, input_seq.shape[0])):
            # Historical data
            ax.plot(lookback_steps, input_seq[series_idx],
                   color=colors[series_idx], linewidth=2, alpha=0.6,
                   label=f'{ticker_names[series_idx]} History')

            # Actual future
            ax.plot(forecast_steps, actual_forecast[series_idx],
                   color=colors[series_idx], linewidth=2.5, alpha=0.8,
                   linestyle='--', marker='o', markersize=4,
                   label=f'{ticker_names[series_idx]} Actual')

            # Predicted future
            ax.plot(forecast_steps, pred[series_idx],
                   color=colors[series_idx], linewidth=2.5, alpha=0.8,
                   linestyle=':', marker='s', markersize=4,
                   label=f'{ticker_names[series_idx]} Predicted')

        # Forecast start line
        ax.axvline(x=0, color='black', linestyle='--', alpha=0.5, linewidth=1.5,
                  label='Forecast Start')

        ax.set_xlabel('Time Steps', fontsize=10)
        ax.set_ylabel('Normalized Price', fontsize=10)
        ax.set_title(f'Test Sample {sample_idx+1}', fontsize=12, fontweight='bold')
        ax.legend(loc='best', fontsize=7, ncol=2)
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('2_time_series_forecasts.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 2_time_series_forecasts.png\n")

# ============================================================================
# 3. DETAILED SINGLE FORECAST WITH ALL ASSETS
# ============================================================================
print("3/5: Creating detailed multi-asset forecast...")

interesting_sample_idx = min(100, len(test_dataset) - 1)
x_sample, y_sample = test_dataset[interesting_sample_idx]
x_sample = x_sample.unsqueeze(0).to(device)

with torch.no_grad():
    pred = model(x_sample).squeeze(0).cpu().numpy()

input_seq = test_dataset[interesting_sample_idx][0].numpy()
actual_forecast = test_dataset[interesting_sample_idx][1].numpy()

n_assets = min(10, input_seq.shape[0])
fig, axes = plt.subplots(5, 2, figsize=(16, 16))
fig.suptitle(f'Detailed Forecast - All Assets (Sample {interesting_sample_idx+1})',
             fontsize=16, fontweight='bold')
axes = axes.flatten()

lookback_steps = range(-input_seq.shape[1], 0)
forecast_steps = range(0, actual_forecast.shape[1])

for asset_idx in range(n_assets):
    ax = axes[asset_idx]

    # Historical
    ax.plot(lookback_steps, input_seq[asset_idx],
           color='gray', linewidth=2, alpha=0.7, label='Historical')

    # Actual
    ax.plot(forecast_steps, actual_forecast[asset_idx],
           color='#e74c3c', linewidth=2.5, marker='o', markersize=5,
           linestyle='--', alpha=0.8, label='Actual')

    # Predicted
    ax.plot(forecast_steps, pred[asset_idx],
           color='#2ecc71', linewidth=2.5, marker='s', markersize=5,
           linestyle=':', alpha=0.8, label='Predicted')

    # Forecast start
    ax.axvline(x=0, color='black', linestyle='--', alpha=0.5, linewidth=1.5)

    # Metrics
    mse = np.mean((pred[asset_idx] - actual_forecast[asset_idx])**2)
    mae = np.mean(np.abs(pred[asset_idx] - actual_forecast[asset_idx]))

    ax.set_xlabel('Time Steps', fontsize=9)
    ax.set_ylabel('Normalized Price', fontsize=9)
    ax.set_title(f'{ticker_names[asset_idx]} | MSE: {mse:.4f} | MAE: {mae:.4f}',
                fontsize=11, fontweight='bold')
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('3_detailed_asset_forecasts.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 3_detailed_asset_forecasts.png\n")

# ============================================================================
# 4. FORECAST ACCURACY HEATMAP
# ============================================================================
print("4/5: Creating forecast accuracy heatmap...")

n_heatmap_samples = min(50, len(test_dataset))
errors_matrix = []

with torch.no_grad():
    for i in range(n_heatmap_samples):
        x_sample, y_sample = test_dataset[i]
        x_sample = x_sample.unsqueeze(0).to(device)
        pred = model(x_sample).squeeze(0).cpu().numpy()

        # Absolute error for each asset
        error = np.abs(pred - y_sample.numpy())
        errors_matrix.append(error.mean(axis=1))

errors_matrix = np.array(errors_matrix).T

fig, ax = plt.subplots(figsize=(14, 8))

im = ax.imshow(errors_matrix, aspect='auto', cmap='RdYlGn_r', interpolation='nearest')

ax.set_xticks(range(0, n_heatmap_samples, 5))
ax.set_xticklabels(range(0, n_heatmap_samples, 5))
ax.set_yticks(range(len(ticker_names)))
ax.set_yticklabels(ticker_names)

ax.set_xlabel('Test Sample Index', fontsize=12)
ax.set_ylabel('Asset', fontsize=12)
ax.set_title('Forecast Error Heatmap (MAE) - Green=Good, Red=Bad',
            fontsize=14, fontweight='bold')

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Mean Absolute Error', fontsize=11)

plt.tight_layout()
plt.savefig('4_forecast_error_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 4_forecast_error_heatmap.png\n")

# ============================================================================
# 5. ROLLING FORECAST PERFORMANCE
# ============================================================================
print("5/5: Creating rolling forecast performance analysis...")

rolling_mse = []
rolling_mae = []

with torch.no_grad():
    for i in range(len(test_dataset)):
        x_sample, y_sample = test_dataset[i]
        x_sample = x_sample.unsqueeze(0).to(device)
        pred = model(x_sample).squeeze(0).cpu().numpy()

        mse = np.mean((pred - y_sample.numpy())**2)
        mae = np.mean(np.abs(pred - y_sample.numpy()))

        rolling_mse.append(mse)
        rolling_mae.append(mae)

# Rolling averages
window_size = 20
rolling_mse_avg = np.convolve(rolling_mse, np.ones(window_size)/window_size, mode='valid')
rolling_mae_avg = np.convolve(rolling_mae, np.ones(window_size)/window_size, mode='valid')

fig, axes = plt.subplots(2, 1, figsize=(14, 8))
fig.suptitle('Rolling Forecast Performance Over Test Set', fontsize=16, fontweight='bold')

# MSE
axes[0].plot(rolling_mse, alpha=0.3, color='lightblue', label='Individual Samples')
axes[0].plot(range(window_size-1, len(rolling_mse)), rolling_mse_avg,
            linewidth=2.5, color='darkblue', label=f'{window_size}-Sample Average')
axes[0].set_ylabel('MSE', fontsize=11)
axes[0].set_title('Mean Squared Error', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(rolling_mae, alpha=0.3, color='lightcoral', label='Individual Samples')
axes[1].plot(range(window_size-1, len(rolling_mae)), rolling_mae_avg,
            linewidth=2.5, color='darkred', label=f'{window_size}-Sample Average')
axes[1].set_xlabel('Test Sample Index', fontsize=11)
axes[1].set_ylabel('MAE', fontsize=11)
axes[1].set_title('Mean Absolute Error', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('5_rolling_performance.png', dpi=300, bbox_inches='tight')
plt.show()
print("   ✓ Saved: 5_rolling_performance.png\n")

# ============================================================================
# SUMMARY
# ============================================================================
print("="*80)
print("✅ ALL VISUALIZATIONS COMPLETE!")
print("="*80)
print("\nGenerated files:")
print("  1. 1_training_dashboard.png       - Training metrics & overfitting analysis")
print("  2. 2_time_series_forecasts.png    - 6 sample forecasts with context")
print("  3. 3_detailed_asset_forecasts.png - All 10 assets for one sample")
print("  4. 4_forecast_error_heatmap.png   - Error patterns across assets & time")
print("  5. 5_rolling_performance.png      - Performance trends over test set")
print("="*80)

print(f"\n{'='*80}")
print(f"FINAL MODEL QUALITY: {quality}")
print(f"Test/Train Ratio: {test_train_ratio:.2f}x")
print(f"{'='*80}\n")